In [1]:
import sys
sys.path.insert(0, '..\..')
sys.path.insert(0, '../..')

In [2]:
from submissions.full_nn.dataLoaders import imageDataLoader
from submissions.full_nn import dataset
from submissions.ab_submission import toolbox

import importlib

importlib.reload(dataset)
importlib.reload(toolbox)

<module 'submissions.ab_submission.toolbox' from '/home/ali/git/follicles_detection/notebooks/full_nn/../../submissions/ab_submission/toolbox.py'>

# Playground

In [3]:
from torch.utils.data import DataLoader

In [4]:
import glob
import pandas as pd
import imageio
from matplotlib import pyplot as plt
import torch

In [5]:
train_files = glob.glob("../../data/train/*.jpg")
test_files = glob.glob("../../data/test/*.jpg")
train_label = pd.read_csv("../../data/train/labels.csv")
test_label = pd.read_csv("../../data/test/labels.csv")

test_loader = imageDataLoader(test_files, test_label[["filename","xmin","xmax","ymin","ymax","label"]])
train_loader = imageDataLoader(train_files, train_label[["filename","xmin","xmax","ymin","ymax","label"]])
predict_loader = imageDataLoader(test_files, None)

In [6]:
import numpy as np
from torch import nn
from torchvision.transforms import Resize, RandomHorizontalFlip, RandomVerticalFlip

In [7]:
def dataloader_collapse (x, resize_ratio=1, random_flip=True):
    """dataloader_collate
    
    Function of collate for the DataLoader class use with the datasetFollicles class.
    This function is used to provide images from the datasetFollicles in a tensor for training task.

    Parameters
    ----------
    x: input data to collate, list containing tuple of a numpy array of size (h,w,3) and a dictionnary is expected
    random_flip: boolean, if True, random flip of the images are performed

    Output
    ------
    Tuple containing a tensor of size :
        (batch_size, *features) with features the size of the features data
        (batch_size, 5) with 5 the one-hot encoding of the label
    """

    Xs = []
    ys = []

    # Compute resize height and width
    height = int(x[0][1]["height"]/resize_ratio)
    width = int(x[0][1]["width"]/resize_ratio)
    h_ratio = height/x[0][1]["height"]
    w_ratio = width/x[0][1]["width"]

    resizer = Resize((height, width))

    if random_flip:
        random_hflip = RandomHorizontalFlip(p=1)
        random_vflip = RandomVerticalFlip(p=1)

    for data in x:
        # Getting data in tensor
        data_tensor = torch.tensor(data[0], dtype=torch.float32)
        data_tensor = torch.moveaxis(data_tensor, 2, 0)

        # Resize image
        data_tensor = resizer(data_tensor)

        # Random transformations
        if random_flip:
            random_flip_number = (np.random.random(2) <= 0.5)
            if random_flip_number[0]:
                data_tensor = random_hflip(data_tensor)
            if random_flip_number[1]:
                data_tensor = random_vflip(data_tensor)

        # Getting label and box

        if data[1]["bbox_label"] is not None:
            ## Computing box
            bbox = np.array(data[1]["bbox"])
            bbox[:,0], bbox[:,2] = bbox[:,0]*w_ratio, bbox[:,2]*w_ratio
            bbox[:,1], bbox[:,3] = bbox[:,1]*h_ratio, bbox[:,3]*h_ratio
            bbox = np.floor(bbox).astype("int")

            ## Fixing box according to flip
            if random_flip:
                if random_flip_number[0]:
                    bbox[:,0], bbox[:,2] = width-bbox[:,2], width-bbox[:,0]
                if random_flip_number[1]:
                    bbox[:,1], bbox[:,3] = height-bbox[:,3], height-bbox[:,1]

            ## Removing negative labels
            label_mask = (np.array(data[1]["bbox_label"]) != 0)

            ## Removing empty box
            not_empty_mask = ((bbox[:,2]-bbox[:,0])*(bbox[:,3]-bbox[:,1])) != 0

            bbox_label = {
                    "boxes":torch.tensor(bbox, dtype=torch.int64)[(label_mask) & (not_empty_mask)],
                    "labels":torch.tensor(data[1]["bbox_label"], dtype=torch.int64)[(label_mask) & (not_empty_mask)]
            }

            ys.append(bbox_label)
        else:
            y = None
        
        Xs.append(data_tensor)

    X = torch.stack(Xs)
    y = ys
    
    return X, y

In [8]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_320_fpn, fasterrcnn_resnet50_fpn

In [9]:
train_dataset = dataset.folliclesDataset(
    train_loader,
    local_path="../../data/train_dataset_fullnn",
    window_size=(1000,1000),
    verbose=False,
    force_reload=False
)

test_dataset = dataset.folliclesDataset(
    test_loader,
    local_path="../../data/test_dataset_fullnn",
    window_size=(1000,1000),
    verbose=False,
    border_condition="ignore",
    force_reload=False
)

In [ ]:
resize_ratio=2

In [15]:
collater = lambda x: dataloader_collapse(x, resize_ratio=resize_ratio, random_flip=True)

In [16]:
def generatePrediction (filename, dataset, resize_factor=4):

    # Getting data
    data = dataset[filename]

    # Resizing
    data_resized = []
    data_resized_ratio = []
    data_metadata = []

    for x in data:
        x_ = torch.tensor(x[0], dtype=torch.float32)/255.

        width = int(x_.shape[0]/resize_factor)
        height = int(x_.shape[1]/resize_factor)
        resizer = Resize((height, width))

        width_ratio = x_.shape[0]/width
        height_ratio = x_.shape[1]/height

        x_ = torch.moveaxis(x_, 2, 0)
        x_ = resizer(x_)

        data_resized.append(x_)
        data_resized_ratio.append((width_ratio, height_ratio))

        data_metadata.append(x[1])
    
    return data_resized, data_resized_ratio, data_metadata

In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collater)

In [22]:
model = fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 5)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=1e-5)

model = model.to("cuda:0")

In [ ]:
# Piste
## Intesect Moving window
## Pretrain breast

In [42]:
n_epochs = 20
print_epoch = 1
print_iter_freq = 100

losses_history = []

for i in range(n_epochs):

    if (i%print_epoch == 0) and (i != 0):
        loss_mean = np.array(epoch_loss).mean(axis=0)
        print(f"Epoch n°{i} - loss_classifier : {loss_mean[0]} - loss_box_reg : {loss_mean[1]} - loss_objectness : {loss_mean[2]} - loss_rpn_box_reg : {loss_mean[3]}")

    if i != 0:
        losses_history += epoch_loss
    
    epoch_loss = []

    iters = 0
    for x, y in train_dataloader:

        model.train()

        optimizer.zero_grad()

        if (iters%print_iter_freq == 0) and (iters != 0):
            loss_mean = np.array(epoch_loss).mean(axis=0)
            print(f"[{i}] loss_classifier : {loss_mean[0]} - loss_box_reg : {loss_mean[1]} - loss_objectness : {loss_mean[2]} - loss_rpn_box_reg : {loss_mean[3]}")

        x = x.to("cuda:0")
        x = x/255.
        for j in range(len(y)):
            for key in y[j].keys():
                y[j][key] = y[j][key].to("cuda:0")

        loss = model(x,y)
        
        losses = sum(loss_ for loss_ in loss.values())
        losses.backward()
        optimizer.step()
        epoch_loss.append([x.detach().cpu().item() for x in loss.values()])

        iters += 1

[0] loss_classifier : 0.1470124568603933 - loss_box_reg : 0.15043586176587267 - loss_objectness : 0.011206137056287844 - loss_rpn_box_reg : 0.002899003131024074


In [33]:
with torch.no_grad():
    model.eval()
    res = model(x)

In [34]:
def show_image_torch(x):
    x_ = x.detach().cpu()
    x_ = torch.moveaxis(x, 0, 2)
    x_ = x_.cpu().numpy()
    x_ = x_.astype("int")

    plt.imshow(x_)

In [35]:
loader = test_loader
dataset_ = test_dataset

files = loader.X_filenames

id_to_class = {
    1:"Primordial",
    2:"Primaire",
    3:"Secondaire",
    4:"Tertiaire"
}

In [39]:
# Prediction

with torch.no_grad():
    model.eval()

    predictions = {}
    for filename in files:
        data = generatePrediction(filename, dataset_, resize_factor=resize_ratio)

        predictions[filename] = []
        for x in data[0]:
            x = x.unsqueeze(dim=0).to("cuda:0")
            y_hat = model(x)[0]

            for key in y_hat.keys():
                y_hat[key] = y_hat[key].detach().cpu().tolist()

            predictions[filename].append(y_hat)

In [40]:
from torchvision.ops import box_iou, nms

In [41]:
for filename in files:
    image_prediction = []
    image_labels = []
    
    print(f"Parsing filename {filename}")

    data = generatePrediction(filename, dataset_, resize_factor=resize_ratio)
    for window, resize_ratio, metadata, pred in zip(*(data + (predictions[filename],))):
        if len(pred["boxes"]) > 0:
            # Getting correct box offsets
            pred_array = np.array(pred["boxes"])
            pred_array[:,0], pred_array[:,2] = resize_ratio[0]*pred_array[:,0], resize_ratio[0]*pred_array[:,2]
            pred_array[:,1], pred_array[:,3] = resize_ratio[1]*pred_array[:,1], resize_ratio[1]*pred_array[:,3]
            
            window_offset_mask = np.repeat([metadata["window_offsets"][0:2]],2, axis=0).flatten()
            pred_array = (window_offset_mask+pred_array).astype("int")
            
            # Merging
            pred_boxes = torch.tensor(pred_array, dtype=torch.float32)
            pred_scores = torch.tensor(pred["scores"], dtype=torch.float32)
            pred_labels = torch.tensor(pred["labels"], dtype=torch.int8)
            indices_nms = nms(
                boxes=pred_boxes,
                scores=pred_scores,
                iou_threshold=0.7
            ) # Check if it works

            pred_boxes = pred_boxes.int()

            image_prediction += pred_boxes[indices_nms,:].tolist()
            image_labels += pred_labels[indices_nms].tolist()

    # Getting image
    preds = [{"bbox":x, "class":id_to_class[y]} for x,y in zip(image_prediction, image_labels)]
    image = loader.get_sample(filename)[0]

    toolbox.write_rectangle(
        image,
        preds,
        folder="../../data/test_predicted_fullnn",
        filename=filename
    )

Parsing filename D-1M06-4.jpg
Parsing filename D-1M06-2.jpg
Parsing filename D-1M06-5.jpg
Parsing filename D-1M06-1.jpg
Parsing filename D-1M06-3.jpg
